In [85]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 as cv
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Activation, MaxPooling2D 
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import normalize
import time

In [86]:
DATADIR = "C:/Users/CALDEN MICHAEL/Desktop/Pure Tensorflow/chest_xray/train"
CATEGORIES = ["NORMAL", "PNEUMONIA"]

IMG_SIZE = 80

training_data = []

def create_training_data():
    for category in CATEGORIES:
        PATH = os.path.join(DATADIR, category)
        category_index = CATEGORIES.index(category)
        for img in os.listdir(PATH):
            try:
                img_array = cv.imread(os.path.join(PATH, img))
                img_array = cv.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([img_array, category_index])
            except Exception as e:
                pass

In [87]:
create_training_data()

In [89]:
random.shuffle(training_data)

In [90]:
X_train = []
Y_train = []
for feature, label in training_data:
    X_train.append(feature)
    Y_train.append(label)

In [91]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [92]:
X_train = X_train / 255.0

In [93]:
conv_layers = [3]
layer_units = [64]
dense_layers = [2]

model = Sequential()

for dense_layer in dense_layers:
    for layer_unit in layer_units:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_unit, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir="loggs/{}".format(NAME))
            print(NAME)
            
            model = Sequential()
            
            model.add(Conv2D(layer_unit, (3, 3), input_shape=X_train.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            
            for l in range(conv_layer - 1):
                model.add(Conv2D(layer_unit, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))
                          
            model.add(Flatten())
                          
            for l in range(dense_layer):
                model.add(Dense(layer_unit))
                model.add(Activation('relu'))
                model.add(Dropout(0.2))
                          
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            model.compile(optimizer=tf.keras.optimizers.Adam(), loss="binary_crossentropy", metrics=['accuracy'])
                          
            model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

            model.save("{}-conv-{}-nodes-{}-dense-{}.model".format(conv_layer, layer_unit, dense_layer, int(time.time())))

3-conv-64-nodes-2-dense-1599061204
Epoch 1/10
147/147 [==============================] - 40s 269ms/step - loss: 0.3948 - accuracy: 0.8334 - val_loss: 0.1649 - val_accuracy: 0.9387
Epoch 2/10
147/147 [==============================] - 39s 262ms/step - loss: 0.1543 - accuracy: 0.9440 - val_loss: 0.0845 - val_accuracy: 0.9655
Epoch 3/10
147/147 [==============================] - 38s 261ms/step - loss: 0.1240 - accuracy: 0.9540 - val_loss: 0.0694 - val_accuracy: 0.9789
Epoch 4/10
147/147 [==============================] - 37s 251ms/step - loss: 0.0921 - accuracy: 0.9661 - val_loss: 0.0680 - val_accuracy: 0.9808
Epoch 5/10
147/147 [==============================] - 37s 252ms/step - loss: 0.0910 - accuracy: 0.9661 - val_loss: 0.0589 - val_accuracy: 0.9808
Epoch 6/10
147/147 [==============================] - 36s 248ms/step - loss: 0.0759 - accuracy: 0.9727 - val_loss: 0.0748 - val_accuracy: 0.9751
Epoch 7/10
147/147 [==============================] - 36s 248ms/step - loss: 0.0724 - accuracy:

In [71]:
print(X_train[0].shape)

(80, 80, 3)


In [88]:
print(len(training_data))

5216
